# Aula 08 - Aprendendo a Ranquear - Exercícios

In [1]:
!pip install "numpy<2"

In [2]:
import pandas as pd
import numpy as np
import random

In [3]:
from surprise import Dataset, Reader, SVD
from collections import defaultdict

In [4]:
!pip install scikit-surprise

### Importar base de dados

In [5]:
!wget https://github.com/mmanzato/MBABigData/raw/master/ml-20m-compact.tar.gz
!tar -xvzf ml-20m-compact.tar.gz

--2025-09-29 17:09:31--  https://github.com/mmanzato/MBABigData/raw/master/ml-20m-compact.tar.gz
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mmanzato/MBABigData/master/ml-20m-compact.tar.gz [following]
--2025-09-29 17:09:31--  https://raw.githubusercontent.com/mmanzato/MBABigData/master/ml-20m-compact.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 65019041 (62M) [application/octet-stream]
Saving to: ‘ml-20m-compact.tar.gz.6’

ml-20m-compact.tar. 100%[===================>]  62.01M   377MB/s    in 0.2s    

2025-09-29 17:09:32 (377 MB/s) - ‘ml-20m-compact.tar.gz.6’ saved [65019041/6501

In [6]:
movies = pd.read_csv('./dataset/movies_sample.csv', names=['itemId', 'title', 'genre'], header=0)
ratings = pd.read_csv('./dataset/ratings_sample.csv', names=['userId', 'itemId', 'rating', 'timestamp'], header=0)
df = ratings[['userId', 'itemId', 'rating']]
df = df.merge(movies[['itemId', 'title']])
df

,userId,itemId,rating,title
0,11,7481,5.0,Enemy Mine (1985)
1,11,1046,4.5,Beautiful Thing (1996)
2,11,616,4.0,"Aristocats, The (1970)"
3,11,3535,2.0,American Psycho (2000)
4,11,5669,5.0,Bowling for Columbine (2002)
...,...,...,...,...
190616,138493,288,5.0,Natural Born Killers (1994)
190617,138493,1748,5.0,Dark City (1998)
190618,138493,616,4.0,"Aristocats, The (1970)"
190619,138493,1597,4.5,Conspiracy Theory (1997)


### Mapeamento de ids

In [7]:
map_users = {user: idx for idx, user in enumerate(df.userId.unique())}
map_items = {item: idx for idx, item in enumerate(df.itemId.unique())}
df['userId'] = df['userId'].map(map_users)
df['itemId'] = df['itemId'].map(map_items)
map_title = {}

for _, row in df.iterrows():
    map_title[row.itemId] = row.title


### Divisão da base em treino e teste

In [8]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=.2, random_state=2)

In [9]:
## Funções que vimmos na aula 8

# Lista de todos os itens que um usuário avalia
def get_item_ids(df, userId):

    if userId not in df['userId'].values:
        return []

    return (df.loc[(df.userId==userId),'itemId'].tolist())

get_item_ids(df, 0)

observed = dict()
unobserved = dict()
all_users = df['userId'].unique().tolist()
all_items = df['itemId'].unique().tolist()

for u in all_users:

    observed[u] = get_item_ids(train, u)
    unobserved[u] = list(set(all_items)-set(observed[u]))

def draw(userId):

    i = random.choice(observed[userId])
    j = random.choice(unobserved[userId])
    return i, j

draw(2)

(18, 407)

In [10]:
# User para a validação
user_id = 1

# o que usuário realmente gostou no teste (ground-truth)
ground_truth = test[test['userId'] == user_id]['itemId'].tolist()

### Nesta aula, você podera escolher entre implementar o primeiro, o segundo ou ambos os exercícios abaixo.

***Exercício 01:*** Comparação entre um algoritmo pair-wise e point-wise:
- Utilizando o BPR implementado em aula, gere uma lista de 10 recomendações para um determinado usuário da base.
- Calcule a precisão média dessas recomendações usando como ground-truth o conjunto de teste.
- Utilize o algoritmo MatrixFactorization (SVD otimizado) de predição de notas para prever as notas dos itens que aquele usuário não avaliou ainda.
- Ordene inversamente esses itens pela nota predita, e recomende os 10 primeiros filmes.
- Calcule a precisão média dessas recomendações e compare o resultado com o resultado obtido pelo BPR.

In [11]:
# Sortear um par visto não visto
def draw(userId):

    i = random.choice(observed[userId])
    j = random.choice(unobserved[userId])
    return i, j

# Função de treinamento do BPRMF
def train_bprmf(train_df, n_factors, lr=0.01, reg=0.05, miter=50):

    n_users = df['userId'].max()+1
    n_items = df['itemId'].max()+1
    item_bias = np.zeros(n_items)
    p = np.random.normal(0, 0.1, (n_users, n_factors))
    q = np.random.normal(0, 0.1, (n_items, n_factors))

    print(f'BPR de range {miter}')

    for t in range(miter):

        random_users = random.choices(train_df['userId'].unique(), k=len(train_df))
        for u in random_users:

            i, j = draw(u)
            x_uij = item_bias[i] - item_bias[j] + (np.dot(p[u], q[i]) - np.dot(p[u], q[j]))
            eps = 1 / (1 + np.exp(x_uij))

            item_bias[i] += lr * (eps - reg * item_bias[i])
            item_bias[j] += lr * (-eps - reg * item_bias[j])

            u_f, i_f, j_f = p[u], q[i], q[j]
            p[u] += lr * ((i_f - j_f) * eps - reg * u_f)
            q[i] += lr * (u_f * eps - reg * i_f)
            q[j] += lr * (-u_f * eps - reg * j_f)

    return item_bias, p, q

# Função para gerar recomendações
def predict_bpr(item_bias, p, q, user_id, N=10):

    w = item_bias.T + np.dot(p, q.T)
    # Ordena os scores para o usuário específico
    scores = w[user_id]
    candidate_items = sorted(range(len(scores)), key=lambda k: scores[k], reverse=True)

    # Filtra itens que o usuário já viu no treino
    partial_ranking = []
    for i in candidate_items:

        if i not in observed[user_id]:
            partial_ranking.append(i)

        if len(partial_ranking) == N:
            break

    return partial_ranking

In [13]:
# BPR
b, p, q = train_bprmf(train, n_factors=15, miter=50)
recommendations_bpr = predict_bpr(b, p, q, user_id, N=10)

# # o que usuário realmente gostou no teste (ground-truth)
# ground_truth = test[test['userId'] == user_id]['itemId'].tolist()

# Precisão@10
hits_bpr = len(set(recommendations_bpr) & set(ground_truth))
precision_bpr = hits_bpr / 10.0

print(f"Recomendações para o usuário {user_id}: {[map_title[i] for i in recommendations_bpr]}")
print(f"Ground Truth: {[map_title[i] for i in ground_truth]}")
print(f"Acertos: {hits_bpr}")
print(f"Precisão@10 (BPR): {precision_bpr:.2f}")

BPR de range 50
Recomendações para o usuário 1: ['Reservoir Dogs (1992)', 'Bowling for Columbine (2002)', 'North by Northwest (1959)', 'Dark City (1998)', 'Karate Kid, The (1984)', "My Best Friend's Wedding (1997)", 'Go (1999)', 'I, Robot (2004)', 'While You Were Sleeping (1995)', 'Rush Hour 2 (2001)']
Ground Truth: ['Sliding Doors (1998)', 'Rush Hour 2 (2001)', 'Go (1999)', 'Reservoir Dogs (1992)', 'Deep End of the Ocean, The (1999)', 'While You Were Sleeping (1995)']
Acertos: 4
Precisão@10 (BPR): 0.40


***Exercicio 02:*** Na aula vimos a implementação do método draw() do algoritmo BPR tradicional. Para um dado usuário, este método retorna aleatoriamente um item que ele viu (**item i**) e um outro item que ele não conhece (**item j**). O algoritmo assume que o item visto é preferível ao que ele não viu, e isso é usado para maximizar a diferença entre os scores desses itens (veja a variável **x_uij** da implementação).

Um problema dessa abordagem é que os itens não vistos são necessariamente encarados como menos relevante pelo algoritmo, o que nem sempre acontece pois pode ser que o usuário não tenha interagido com aquele item pois não o conhece, mas que poderia gostar. Um outro problema é quando ambos os itens i e j foram vistos pelo usuário, o que numa estratégia com feedback exclusivamente implícito o BPR não consegue diferenciar qual item é preferível ao usuário.

Você consegue pensar numa estratégia aperfeiçoada para o método draw()? Exemplos:
- Se tivermos acesso aos metadados, podemos construir um perfil para o usuário de modo que o método draw() vai retornar itens j que estão mais distantes desse perfil.
- Se dois itens i e j foram vistos, use os metadados (ou as notas, caso use feedback explícito) para decidir qual deles deve ser o i e qual deve ser o j.
- Etc.

Implemente pelo menos uma estratégia de aperfeiçoamento do BPR, e compare com a versão original.

In [18]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(train[['userId', 'itemId', 'rating']], reader)
trainset = data.build_full_trainset()

# SVD
svd = SVD(n_factors=100, n_epochs=100, random_state=2)
svd.fit(trainset)

# recomendações do mesmo usuário
unobserved_items_svd = unobserved[user_id]
predictions = [svd.predict(user_id, item_id) for item_id in unobserved_items_svd]

# Ordenar pela nota
recommendations_svd_obj = sorted(predictions, key=lambda x: x.est, reverse=True)[:10]
recommendations_svd = [pred.iid for pred in recommendations_svd_obj]

# Precisão@10 para o SVD
hits_svd = len(set(recommendations_svd) & set(ground_truth))
precision_svd = hits_svd / 10.0

print(f"Recomendações para o usuário {user_id}: {[map_title[i] for i in recommendations_svd]}")
print(f"Ground Truth: {[map_title[i] for i in ground_truth]}")
print(f"Acertos: {hits_svd}")
print(f"Precisão@10 (SVD): {precision_svd:.2f}")

Recomendações para o usuário 1: ['Balance (1989)', 'Love Streams (1984)', 'Into the Woods (1991)', 'Dark City (1998)', 'Earrings of Madame de..., The (Madame de...) (1953)', 'Reservoir Dogs (1992)', "Muriel, or The Time of Return (Muriel ou Le temps d'un retour) (1963)", 'Hearts and Minds (1974)', 'Diamond Men (2001)', 'Samouraï, Le (Godson, The) (1967)']
Ground Truth: ['Sliding Doors (1998)', 'Rush Hour 2 (2001)', 'Go (1999)', 'Reservoir Dogs (1992)', 'Deep End of the Ocean, The (1999)', 'While You Were Sleeping (1995)']
Acertos: 1
Precisão@10 (SVD): 0.10
